# LSTM 

## Outline
1. [Imports](#imports)
2. [Data](#data)
3. [Model](#model)
5. [Train](#train)
6. [Predict](#predict)

<a name=imports></a>
## Imports

In [50]:
%load_ext autoreload
%autoreload 2
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

import preprocessing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a name=data></a>
## Data

### Ingestion 

In [51]:
# Check preprocessing.py for implementation
(india_cases_df, india_vacc_df, usa_cases_df, usa_vacc_df) = preprocessing.ingestion()

Parent Directory /covid19-prediction/models
Path: /covid19-prediction/models/../cleaned_datasets/india/daily_cases_india.csv
Exists: True
India Cases:
          Date  Confirmed  Deaths  Recovered
0  2020-01-30        NaN     NaN        NaN
1  2020-01-31        0.0     0.0        0.0
2  2020-02-01        0.0     0.0        0.0
3  2020-02-02        1.0     0.0        0.0
4  2020-02-03        1.0     0.0        0.0 

India Vacc:
          Date  Total_Doses
0  2021-01-15          NaN
1  2021-01-16          NaN
2  2021-01-17      33120.0
3  2021-01-18     229748.0
4  2021-01-19     220786.0 

USA Cases:
          Date  Confirmed  Deaths  Recovered
0  2020-04-12        NaN     NaN        NaN
1  2020-04-13    25255.0  1542.0    11785.0
2  2020-04-14    26694.0  2327.0     6484.0
3  2020-04-15    29318.0  2474.0     6093.0
4  2020-04-16    31235.0  4607.0     5234.0 

USA Vacc:
          Date  Total_Doses
0  2020-12-20     556208.0
1  2020-12-21     614117.0
2  2020-12-22     811071.0
3  2020-

### Pre-Processing

### Multivariate

In [52]:
# Check preprocessing.py for implementation
# Number of Null in India - 563
# Number of Null in USA - 261
(india_multi, usa_multi) = preprocessing.multivariate(india_cases_df, india_vacc_df, usa_cases_df, usa_vacc_df)

India:
          Date  Confirmed  Total_Doses
0  2020-01-30        NaN          NaN
1  2020-01-31        0.0          NaN
2  2020-02-01        0.0          NaN
3  2020-02-02        1.0          NaN
4  2020-02-03        1.0          NaN
USA:
          Date  Confirmed  Total_Doses
0  2020-04-12        NaN          NaN
1  2020-04-13    25255.0          NaN
2  2020-04-14    26694.0          NaN
3  2020-04-15    29318.0          NaN
4  2020-04-16    31235.0          NaN


### Univariate 

In [53]:
# Check preprocessing.py for implementation
# Number of Null in India - 0
# Number of Null in USA - 0
(india_cases_uni, usa_cases_uni) = preprocessing.univariate(india_cases_df, usa_cases_df)

India Cases:
    Confirmed
1        0.0
2        0.0
3        1.0
4        1.0
5        0.0 

USA Cases:
    Confirmed
1    25255.0
2    26694.0
3    29318.0
4    31235.0
5    32175.0 



### Drop Null

In [54]:
(india_multi, usa_multi) = preprocessing.dropNull(india_multi, usa_multi)

India:
            Date  Confirmed  Total_Doses
353  2021-01-17    13788.0      33120.0
354  2021-01-18    10050.0     229748.0
355  2021-01-19    13816.0     220786.0
356  2021-01-20    15244.0     131649.0
357  2021-01-21    14545.0     237050.0
USA:
            Date  Confirmed  Total_Doses
252  2020-12-20   185355.0     556208.0
253  2020-12-21   189716.0     614117.0
254  2020-12-22   201947.0     811071.0
255  2020-12-23   225777.0    1008025.0
256  2020-12-24   204900.0    1476305.0


### Normalize

In [55]:
# Check preprocessing.py for implementation
(india_uni_norm, usa_uni_norm, india_multi_norm, usa_multi_norm,india_cases_mean,india_cases_std,usa_cases_mean,usa_cases_std,india_multi_mean,india_multi_std,
usa_multi_std) = preprocessing.normalize(india_cases_uni, 
                                                                                           usa_cases_uni, 
                                                                                           india_multi, 
                                                                                           usa_multi)

India Cases univariate:
    Confirmed
1  -0.661747
2  -0.661747
3  -0.661734
4  -0.661734
5  -0.661747 

USA Cases univariate:
    Confirmed
1  -0.596456
2  -0.584433
3  -0.562508
4  -0.546491
5  -0.538637 

India Cases multivariate:
      Confirmed  Total_Doses
353  -0.562592    -1.302360
354  -0.601182    -1.243797
355  -0.562303    -1.246466
356  -0.547561    -1.273014
357  -0.554777    -1.241622 

USA Cases multivariate:
      Confirmed  Total_Doses
252   0.492914    -1.809935
253   0.523316    -1.809749
254   0.608584    -1.809118
255   0.774713    -1.808487
256   0.629171    -1.806987 



### Create Splits

In [56]:
# Create train test splits
(india_uni_train, india_uni_test, usa_uni_train, usa_uni_test) = preprocessing.split(india_uni_norm, usa_uni_norm, 0.2)
(india_multi_train, india_multi_test, 
 usa_multi_train, usa_multi_test) = preprocessing.split(india_multi_norm, usa_multi_norm, 0.2)

India:
      Confirmed
1    -0.661747
2    -0.661747
3    -0.661734
4    -0.661734
5    -0.661747
..         ...
565  -0.199747
566  -0.183686
567  -0.181453
568  -0.209217
569  -0.255301

[569 rows x 1 columns] 

USA:
      Confirmed
1    -0.596456
2    -0.584433
3    -0.562508
4    -0.546491
5    -0.538637
..         ...
507   0.882933
508   0.655423
509   1.161010
510  -0.188293
511  -0.412010

[511 rows x 1 columns] 

India:
      Confirmed  Total_Doses
353  -0.562592    -1.302360
354  -0.601182    -1.243797
355  -0.562303    -1.246466
356  -0.547561    -1.273014
357  -0.554777    -1.241622
..         ...          ...
638  -0.557172     1.442780
639  -0.572482     0.707335
640  -0.575745    -0.078616
641  -0.597332     0.183795
642  -0.582053    -0.697398

[279 rows x 2 columns] 

USA:
      Confirmed  Total_Doses
252   0.492914    -1.809935
253   0.523316    -1.809749
254   0.608584    -1.809118
255   0.774713    -1.808487
256   0.629171    -1.806987
..         ...          ...
55

### Data Transformation
Simple conversion from pandas dataframe to numpy array by extracting the values

In [57]:
# Univariate
india_uni_train_np, india_uni_test_np = india_uni_train.values,  india_uni_test.values
usa_uni_train_np, usa_uni_test_np = usa_uni_train.values, usa_uni_test.values
# Multivariate
india_multi_train_np, india_multi_test_np = india_multi_train.values,  india_multi_test.values
usa_multi_train_np, usa_multi_test_np = usa_multi_train.values, usa_multi_test.values

In [58]:
print(india_uni_train_np.shape, india_uni_test_np.shape)
print(india_multi_train_np.shape, india_multi_test_np.shape)

(569, 1) (143, 1)
(279, 2) (70, 2)


### Reshape
Converts the exiting train data set to match the input parameters of an LSTM (Samples, Time Steps, Features). The `lstm_data_transform()` method takes in 3 arguments: X data, y data and number of features.

We will be performing autoregression on the univariate data, therefore the X and y values are the same. As for the multivariate data, we will be using the confirmed column (index 0) as the y value.

In [59]:
timesteps = 3

In [60]:
# Univariate
india_uni_train_X, india_uni_train_y = preprocessing.lstm_data_transform(india_uni_train_np, india_uni_train_np, timesteps)
usa_uni_train_X, usa_uni_train_y = preprocessing.lstm_data_transform(usa_uni_train_np, usa_uni_train_np, timesteps)
# Multivariate
india_multi_train_X, india_multi_train_y = preprocessing.lstm_data_transform(india_multi_train_np, india_multi_train_np[:,0], timesteps)
usa_multi_train_X, usa_multi_train_y = preprocessing.lstm_data_transform(usa_multi_train_np, usa_multi_train_np[:,0], timesteps)

In [61]:
print("Univariate India shape:",india_uni_train_X.shape, india_uni_train_y.shape)
print("Multivariate India shape:",india_multi_train_X.shape, india_multi_train_y.shape)

Univariate India shape: (566, 3, 1) (566, 1)
Multivariate India shape: (276, 3, 2) (276,)


The univariate array shape for the India dataset has 566 samples, 3 as the time step, and 1 feature for the X values. The y values is just the 566 samples.

Similarly, the multivariate array shape for the India dataset has 276 samples, 3 as the time step, and 2 features for the X values. The y values is just the 276 samples of the 0th column (Confirmed cases).

<a name=model></a>
## Model
We will define 4 models in total. One for the univariate data and another for the multivariate for each country.

In [62]:
import LSTMmodels
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

In [63]:
univariate_model_ind = LSTMmodels.build_lstm(time_steps=india_uni_train_X.shape[1], features=india_uni_train_X.shape[2])
univariate_model_usa = LSTMmodels.build_lstm(time_steps=usa_uni_train_X.shape[1], features=usa_uni_train_X.shape[2])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [64]:
multivariate_model_ind = LSTMmodels.build_lstm(time_steps=india_multi_train_X.shape[1], features=india_multi_train_X.shape[2])
multivariate_model_usa = LSTMmodels.build_lstm(time_steps=usa_multi_train_X.shape[1], features=usa_multi_train_X.shape[2])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100)               41200     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 41,301
Trainable params: 41,301
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               41200     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 41,301
Trainable params: 41,301
Non-trainable params: 0
_________________________________________________________________


<a name=train></a>
## Train 

In [65]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
# Tensorboard callback
logdir = os.path.join(parentDir+"/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [66]:
def train_models(model,train_X,train_y,test_X,test_y):
    model.fit(train_X,train_y,
              validation_data=(test_X,test_y),
              epochs=300,
              callbacks=[tensorboard_callback])
    return model
#     model.save(save_name)

In [67]:
# Train the model on univariate India dataset 
univariate_model_ind = LSTMmodels.train_model(univariate_model_ind,india_uni_train_X,india_uni_train_y,tensorboard_callback)

Epoch 1/300
18/18 [==============================] - 1s 21ms/step - loss: 0.5534 - mae: 0.5534
Epoch 2/300
18/18 [==============================] - 0s 5ms/step - loss: 0.3821 - mae: 0.3821
Epoch 3/300
18/18 [==============================] - 0s 5ms/step - loss: 0.1517 - mae: 0.1517
Epoch 4/300
18/18 [==============================] - 0s 4ms/step - loss: 0.1004 - mae: 0.1004
Epoch 5/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0965 - mae: 0.0965
Epoch 6/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0937 - mae: 0.0937
Epoch 7/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0886 - mae: 0.0886
Epoch 8/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0917 - mae: 0.0917
Epoch 9/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0878 - mae: 0.0878
Epoch 10/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0883 - mae: 0.0883
Epoch 11/300
18/18 [==============================] - 0s 4

18/18 [==============================] - 0s 4ms/step - loss: 0.0682 - mae: 0.0682
Epoch 88/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0661 - mae: 0.0661
Epoch 89/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0694 - mae: 0.0694
Epoch 90/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0709 - mae: 0.0709
Epoch 91/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0660 - mae: 0.0660
Epoch 92/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0684 - mae: 0.0684
Epoch 93/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0669 - mae: 0.0669
Epoch 94/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0693 - mae: 0.0693
Epoch 95/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0679 - mae: 0.0679
Epoch 96/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0681 - mae: 0.0681
Epoch 97/300
18/18 [==============================] - 0s 5ms/st

18/18 [==============================] - 0s 4ms/step - loss: 0.0594 - mae: 0.0594
Epoch 173/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0593 - mae: 0.0593
Epoch 174/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0579 - mae: 0.0579
Epoch 175/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0566 - mae: 0.0566
Epoch 176/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0585 - mae: 0.0585
Epoch 177/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0611 - mae: 0.0611
Epoch 178/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0600 - mae: 0.0600
Epoch 179/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0565 - mae: 0.0565
Epoch 180/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0599 - mae: 0.0599
Epoch 181/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0581 - mae: 0.0581
Epoch 182/300
18/18 [==============================] -

18/18 [==============================] - 0s 5ms/step - loss: 0.0525 - mae: 0.0525
Epoch 258/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0520 - mae: 0.0520
Epoch 259/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0509 - mae: 0.0509
Epoch 260/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0529 - mae: 0.0529
Epoch 261/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0494 - mae: 0.0494
Epoch 262/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0511 - mae: 0.0511
Epoch 263/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0517 - mae: 0.0517
Epoch 264/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0499 - mae: 0.0499
Epoch 265/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0508 - mae: 0.0508
Epoch 266/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0503 - mae: 0.0503
Epoch 267/300
18/18 [==============================] -

In [71]:
# Train the model on univariate USA dataset 
univariate_model_usa = LSTMmodels.train_model(univariate_model_usa,usa_uni_train_X,usa_uni_train_y,tensorboard_callback)

Epoch 1/300
16/16 [==============================] - 1s 20ms/step - loss: 0.4601 - mae: 0.4601
Epoch 2/300
16/16 [==============================] - 0s 5ms/step - loss: 0.3488 - mae: 0.3488
Epoch 3/300
16/16 [==============================] - 0s 5ms/step - loss: 0.2187 - mae: 0.2187
Epoch 4/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1503 - mae: 0.1503
Epoch 5/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1454 - mae: 0.1454
Epoch 6/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1402 - mae: 0.1402
Epoch 7/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1396 - mae: 0.1396
Epoch 8/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1392 - mae: 0.1392
Epoch 9/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1392 - mae: 0.1392
Epoch 10/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1381 - mae: 0.1381
Epoch 11/300
16/16 [==============================] - 0s 4

16/16 [==============================] - 0s 5ms/step - loss: 0.1085 - mae: 0.1085
Epoch 88/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1081 - mae: 0.1081
Epoch 89/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1083 - mae: 0.1083
Epoch 90/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1106 - mae: 0.1106
Epoch 91/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1084 - mae: 0.1084
Epoch 92/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1071 - mae: 0.1071
Epoch 93/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1072 - mae: 0.1072
Epoch 94/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1074 - mae: 0.1074
Epoch 95/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1075 - mae: 0.1075
Epoch 96/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1081 - mae: 0.1081
Epoch 97/300
16/16 [==============================] - 0s 5ms/st

16/16 [==============================] - 0s 5ms/step - loss: 0.1043 - mae: 0.1043
Epoch 173/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1032 - mae: 0.1032
Epoch 174/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1040 - mae: 0.1040
Epoch 175/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1025 - mae: 0.1025
Epoch 176/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1026 - mae: 0.1026
Epoch 177/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1016 - mae: 0.1016
Epoch 178/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1038 - mae: 0.1038
Epoch 179/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1027 - mae: 0.1027
Epoch 180/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1032 - mae: 0.1032
Epoch 181/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1027 - mae: 0.1027
Epoch 182/300
16/16 [==============================] -

16/16 [==============================] - 0s 5ms/step - loss: 0.0995 - mae: 0.0995
Epoch 258/300
16/16 [==============================] - 0s 4ms/step - loss: 0.0992 - mae: 0.0992
Epoch 259/300
16/16 [==============================] - 0s 4ms/step - loss: 0.0984 - mae: 0.0984
Epoch 260/300
16/16 [==============================] - 0s 4ms/step - loss: 0.0989 - mae: 0.0989
Epoch 261/300
16/16 [==============================] - 0s 4ms/step - loss: 0.0994 - mae: 0.0994
Epoch 262/300
16/16 [==============================] - 0s 4ms/step - loss: 0.0977 - mae: 0.0977
Epoch 263/300
16/16 [==============================] - 0s 4ms/step - loss: 0.0981 - mae: 0.0981
Epoch 264/300
16/16 [==============================] - 0s 5ms/step - loss: 0.1001 - mae: 0.1001
Epoch 265/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1000 - mae: 0.1000
Epoch 266/300
16/16 [==============================] - 0s 4ms/step - loss: 0.1000 - mae: 0.1000
Epoch 267/300
16/16 [==============================] -

In [20]:
# Train the model on multivariate India dataset
multivariate_model_ind = LSTMmodels.train_model(multivariate_model_ind,india_multi_train_X,india_multi_train_y,tensorboard_callback)

Epoch 1/300
9/9 [==============================] - 1s 33ms/step - loss: 0.6297 - mae: 0.6297
Epoch 2/300
9/9 [==============================] - 0s 5ms/step - loss: 0.5118 - mae: 0.5118
Epoch 3/300
9/9 [==============================] - 0s 5ms/step - loss: 0.3822 - mae: 0.3822
Epoch 4/300
9/9 [==============================] - 0s 5ms/step - loss: 0.2640 - mae: 0.2640
Epoch 5/300
9/9 [==============================] - 0s 5ms/step - loss: 0.1416 - mae: 0.1416
Epoch 6/300
9/9 [==============================] - 0s 5ms/step - loss: 0.1256 - mae: 0.1256
Epoch 7/300
9/9 [==============================] - 0s 5ms/step - loss: 0.1106 - mae: 0.1106
Epoch 8/300
9/9 [==============================] - 0s 5ms/step - loss: 0.1085 - mae: 0.1085
Epoch 9/300
9/9 [==============================] - 0s 5ms/step - loss: 0.1019 - mae: 0.1019
Epoch 10/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0989 - mae: 0.0989
Epoch 11/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0963

9/9 [==============================] - 0s 5ms/step - loss: 0.0649 - mae: 0.0649
Epoch 90/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0616 - mae: 0.0616
Epoch 91/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0646 - mae: 0.0646
Epoch 92/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0614 - mae: 0.0614
Epoch 93/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0605 - mae: 0.0605
Epoch 94/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0627 - mae: 0.0627
Epoch 95/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0621 - mae: 0.0621
Epoch 96/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0596 - mae: 0.0596
Epoch 97/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0594 - mae: 0.0594
Epoch 98/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0582 - mae: 0.0582
Epoch 99/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0581 - ma

9/9 [==============================] - 0s 4ms/step - loss: 0.0446 - mae: 0.0446
Epoch 177/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0455 - mae: 0.0455
Epoch 178/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0463 - mae: 0.0463
Epoch 179/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0448 - mae: 0.0448
Epoch 180/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0459 - mae: 0.0459
Epoch 181/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0449 - mae: 0.0449
Epoch 182/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0494 - mae: 0.0494
Epoch 183/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0512 - mae: 0.0512
Epoch 184/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0540 - mae: 0.0540
Epoch 185/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0484 - mae: 0.0484
Epoch 186/300
9/9 [==============================] - 0s 5ms/step - loss: 0

9/9 [==============================] - 0s 5ms/step - loss: 0.0433 - mae: 0.0433
Epoch 264/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0442 - mae: 0.0442
Epoch 265/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0428 - mae: 0.0428
Epoch 266/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0448 - mae: 0.0448
Epoch 267/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0461 - mae: 0.0461
Epoch 268/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0452 - mae: 0.0452
Epoch 269/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0450 - mae: 0.0450
Epoch 270/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0435 - mae: 0.0435
Epoch 271/300
9/9 [==============================] - 0s 4ms/step - loss: 0.0434 - mae: 0.0434
Epoch 272/300
9/9 [==============================] - 0s 5ms/step - loss: 0.0441 - mae: 0.0441
Epoch 273/300
9/9 [==============================] - 0s 4ms/step - loss: 0

In [21]:
# Train the model on multivariate USA dataset
multivariate_model_usa = LSTMmodels.train_model(multivariate_model_usa,usa_multi_train_X,usa_multi_train_y,tensorboard_callback)

Epoch 1/300
10/10 [==============================] - 1s 29ms/step - loss: 0.4584 - mae: 0.4584
Epoch 2/300
10/10 [==============================] - 0s 4ms/step - loss: 0.4158 - mae: 0.4158
Epoch 3/300
10/10 [==============================] - 0s 4ms/step - loss: 0.3699 - mae: 0.3699
Epoch 4/300
10/10 [==============================] - 0s 4ms/step - loss: 0.3087 - mae: 0.3087
Epoch 5/300
10/10 [==============================] - 0s 4ms/step - loss: 0.2421 - mae: 0.2421
Epoch 6/300
10/10 [==============================] - 0s 5ms/step - loss: 0.2141 - mae: 0.2141
Epoch 7/300
10/10 [==============================] - 0s 5ms/step - loss: 0.2071 - mae: 0.2071
Epoch 8/300
10/10 [==============================] - 0s 4ms/step - loss: 0.2027 - mae: 0.2027
Epoch 9/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1953 - mae: 0.1953
Epoch 10/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1899 - mae: 0.1899
Epoch 11/300
10/10 [==============================] - 0s 4

10/10 [==============================] - 0s 5ms/step - loss: 0.1299 - mae: 0.1299
Epoch 88/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1272 - mae: 0.1272
Epoch 89/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1269 - mae: 0.1269
Epoch 90/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1270 - mae: 0.1270
Epoch 91/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1260 - mae: 0.1260
Epoch 92/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1297 - mae: 0.1297
Epoch 93/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1289 - mae: 0.1289
Epoch 94/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1252 - mae: 0.1252
Epoch 95/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1253 - mae: 0.1253
Epoch 96/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1252 - mae: 0.1252
Epoch 97/300
10/10 [==============================] - 0s 4ms/st

10/10 [==============================] - 0s 4ms/step - loss: 0.1208 - mae: 0.1208
Epoch 173/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1187 - mae: 0.1187
Epoch 174/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1225 - mae: 0.1225
Epoch 175/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1220 - mae: 0.1220
Epoch 176/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1225 - mae: 0.1225
Epoch 177/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1214 - mae: 0.1214
Epoch 178/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1245 - mae: 0.1245
Epoch 179/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1244 - mae: 0.1244
Epoch 180/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1182 - mae: 0.1182
Epoch 181/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1181 - mae: 0.1181
Epoch 182/300
10/10 [==============================] -

10/10 [==============================] - 0s 5ms/step - loss: 0.1140 - mae: 0.1140
Epoch 258/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1133 - mae: 0.1133
Epoch 259/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1103 - mae: 0.1103
Epoch 260/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1078 - mae: 0.1078
Epoch 261/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1082 - mae: 0.1082
Epoch 262/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1058 - mae: 0.1058
Epoch 263/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1067 - mae: 0.1067
Epoch 264/300
10/10 [==============================] - 0s 5ms/step - loss: 0.1076 - mae: 0.1076
Epoch 265/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1065 - mae: 0.1065
Epoch 266/300
10/10 [==============================] - 0s 4ms/step - loss: 0.1072 - mae: 0.1072
Epoch 267/300
10/10 [==============================] -

<a name=predict></a>
## Predict

In [72]:
import evaluate

In [73]:
india_uni_test_np.shape

(143, 1)

In [74]:
india_multi_test_np.shape

(70, 2)

In [75]:
# Univariate
yhat_uni_india = evaluate.make_predictions(univariate_model_ind,india_uni_test_np,timesteps,india_uni_test_np.shape[1])
yhat_uni_usa = evaluate.make_predictions(univariate_model_usa,usa_uni_test_np,timesteps,usa_uni_test_np.shape[1])

hello
itr size: 140
X_data shape (1, 3, 1)
X_data: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]]]
X_data input: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]]]
forecast: [[[-0.16707543]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]]]
forecast: [[[-0.17576547]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]]]
forecast: [[[-0.18560839]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]]]
forecast: [[[-0.19292518]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]]]
forecast: [[[-0.19781642]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]
  [-0.19781642]]]
forecast: [[[-0.20141515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]
  [-0.19781

forecast: [[[-0.21122283]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]
  [-0.19781642]
  [-0.20141515]
  [-0.20404013]
  [-0.2059484 ]
  [-0.20742874]
  [-0.20854589]
  [-0.2093226 ]
  [-0.20994948]
  [-0.21039592]
  [-0.21076989]
  [-0.21099085]
  [-0.2110227 ]
  [-0.21113786]
  [-0.21123894]
  [-0.2111882 ]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]]]
forecast: [[[-0.21122283]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]
  [-0.19781642]
  [-0.20141515]
  [-0.20404013]
  [-0.2059484 ]
  [-0.20742874]
  [-0.20854589]
  [-0.2093226 ]
  [-0.20994948]
  [-0.21039592]
  [-0.21076989]
  [-0.21099085]
  [-0.211022

forecast: [[[-0.21122283]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]
  [-0.19781642]
  [-0.20141515]
  [-0.20404013]
  [-0.2059484 ]
  [-0.20742874]
  [-0.20854589]
  [-0.2093226 ]
  [-0.20994948]
  [-0.21039592]
  [-0.21076989]
  [-0.21099085]
  [-0.2110227 ]
  [-0.21113786]
  [-0.21123894]
  [-0.2111882 ]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]]]
forecast: [[[-0.2

forecast: [[[-0.21122283]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]
  [-0.19781642]
  [-0.20141515]
  [-0.20404013]
  [-0.2059484 ]
  [-0.20742874]
  [-0.20854589]
  [-0.2093226 ]
  [-0.20994948]
  [-0.21039592]
  [-0.21076989]
  [-0.21099085]
  [-0.2110227 ]
  [-0.21113786]
  [-0.21123894]
  [-0.2111882 ]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [

forecast: [[[-0.21122283]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.16707543]
  [-0.17576547]
  [-0.18560839]
  [-0.19292518]
  [-0.19781642]
  [-0.20141515]
  [-0.20404013]
  [-0.2059484 ]
  [-0.20742874]
  [-0.20854589]
  [-0.2093226 ]
  [-0.20994948]
  [-0.21039592]
  [-0.21076989]
  [-0.21099085]
  [-0.2110227 ]
  [-0.21113786]
  [-0.21123894]
  [-0.2111882 ]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [-0.21122283]
  [

...

forecast: [[[1.0376683]]]
In loop: [[[-0.13399099]
  [ 1.32147655]
  [ 0.72184003]
  [ 0.87541145]
  [ 0.87929738]
  [ 0.95844901]
  [ 1.02187121]
  [ 1.06405568]
  [ 1.08102167]
  [ 1.06960857]
  [ 1.04518116]
  [ 1.03067887]
  [ 1.02915657]
  [ 1.03200901]
  [ 1.03678501]
  [ 1.04029715]
  [ 1.04062724]
  [ 1.03978074]
  [ 1.03792691]
  [ 1.03642261]
  [ 1.0354681 ]
  [ 1.03566635]
  [ 1.03732383]
  [ 1.03793132]
  [ 1.03831279]
  [ 1.03779495]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1

In loop: [[[-0.13399099]
  [ 1.32147655]
  [ 0.72184003]
  [ 0.87541145]
  [ 0.87929738]
  [ 0.95844901]
  [ 1.02187121]
  [ 1.06405568]
  [ 1.08102167]
  [ 1.06960857]
  [ 1.04518116]
  [ 1.03067887]
  [ 1.02915657]
  [ 1.03200901]
  [ 1.03678501]
  [ 1.04029715]
  [ 1.04062724]
  [ 1.03978074]
  [ 1.03792691]
  [ 1.03642261]
  [ 1.0354681 ]
  [ 1.03566635]
  [ 1.03732383]
  [ 1.03793132]
  [ 1.03831279]
  [ 1.03779495]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]

forecast: [[[1.0376683]]]
In loop: [[[-0.13399099]
  [ 1.32147655]
  [ 0.72184003]
  [ 0.87541145]
  [ 0.87929738]
  [ 0.95844901]
  [ 1.02187121]
  [ 1.06405568]
  [ 1.08102167]
  [ 1.06960857]
  [ 1.04518116]
  [ 1.03067887]
  [ 1.02915657]
  [ 1.03200901]
  [ 1.03678501]
  [ 1.04029715]
  [ 1.04062724]
  [ 1.03978074]
  [ 1.03792691]
  [ 1.03642261]
  [ 1.0354681 ]
  [ 1.03566635]
  [ 1.03732383]
  [ 1.03793132]
  [ 1.03831279]
  [ 1.03779495]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1

forecast: [[[1.0376683]]]
In loop: [[[-0.13399099]
  [ 1.32147655]
  [ 0.72184003]
  [ 0.87541145]
  [ 0.87929738]
  [ 0.95844901]
  [ 1.02187121]
  [ 1.06405568]
  [ 1.08102167]
  [ 1.06960857]
  [ 1.04518116]
  [ 1.03067887]
  [ 1.02915657]
  [ 1.03200901]
  [ 1.03678501]
  [ 1.04029715]
  [ 1.04062724]
  [ 1.03978074]
  [ 1.03792691]
  [ 1.03642261]
  [ 1.0354681 ]
  [ 1.03566635]
  [ 1.03732383]
  [ 1.03793132]
  [ 1.03831279]
  [ 1.03779495]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1

forecast: [[[1.0376683]]]
In loop: [[[-0.13399099]
  [ 1.32147655]
  [ 0.72184003]
  [ 0.87541145]
  [ 0.87929738]
  [ 0.95844901]
  [ 1.02187121]
  [ 1.06405568]
  [ 1.08102167]
  [ 1.06960857]
  [ 1.04518116]
  [ 1.03067887]
  [ 1.02915657]
  [ 1.03200901]
  [ 1.03678501]
  [ 1.04029715]
  [ 1.04062724]
  [ 1.03978074]
  [ 1.03792691]
  [ 1.03642261]
  [ 1.0354681 ]
  [ 1.03566635]
  [ 1.03732383]
  [ 1.03793132]
  [ 1.03831279]
  [ 1.03779495]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1.03766835]
  [ 1

In [26]:
# Multivariate
yhat_multi_india = evaluate.make_predictions(multivariate_model_ind,india_multi_test_np,timesteps,india_multi_test_np.shape[1])
yhat_multi_usa = evaluate.make_predictions(multivariate_model_usa,usa_multi_test_np,timesteps,usa_multi_test_np.shape[1])

hello
itr size: 137
X_data shape (1, 3, 2)
X_data: [[[-0.57191463  0.27844561]
  [-0.57352513 -0.70228723]
  [-0.59210778 -1.10781997]]]
X_data input: [[[-0.57191463  0.27844561]
  [-0.57352513 -0.70228723]
  [-0.59210778 -1.10781997]]]
forecast: [[[-0.59379756]]]


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 2, the array at index 0 has size 2 and the array at index 1 has size 1

In [ ]:
yhat_uni_india.shape

## Evaluate

In [76]:
from tensorflow.keras.losses import MeanAbsolutePercentageError, MeanAbsoluteError

### Denormalize Data 

In [85]:
type(np.array(india_cases_mean))
np.array(india_cases_mean)

array([50387.34410112])

In [86]:
# Denormalize univariate India data
india_uni_test_np_denorm = preprocessing.de_normalize(india_uni_test_np.flatten()[timesteps:],np.array(india_cases_mean),np.array(india_cases_std))
yhat_uni_india_denorm = preprocessing.de_normalize(yhat_uni_india.flatten()[timesteps:],np.array(india_cases_mean),np.array(india_cases_std))

In [95]:
# Denormalize univariate USA data
usa_uni_test_np_denorm = preprocessing.de_normalize(usa_uni_test_np.flatten()[timesteps:],np.array(usa_cases_mean),np.array(usa_cases_std))
yhat_uni_usa_denorm = preprocessing.de_normalize(yhat_uni_usa.flatten()[timesteps:],np.array(usa_cases_mean),np.array(usa_cases_std))

### Get Metrics 
Return format: (mape, mae, rmse)

In [104]:
uni_india_eval = evaluate.evaluate(india_uni_test_np.flatten()[timesteps:], yhat_uni_india.flatten()[timesteps:])
uni_india_eval.insert(0,'Univariate India')
uni_india_eval

['Univariate India', 64.37289428710938, 0.2840528190135956, 0.3981384]

In [105]:
uni_usa_eval = evaluate.evaluate(usa_uni_test_np.flatten()[timesteps:], yhat_uni_usa.flatten()[timesteps:])
uni_usa_eval.insert(0,'Univariate USA')

In [ ]:
multi_india_eval = evaluate.evaluate(india_multi_test_np.flatten()[timesteps:], yhat_multi_india.flatten()[timesteps:])
multi_india_eval

### Visualize performance

In [107]:
d = [uni_india_eval,uni_usa_eval]
pd.DataFrame(data=d, columns=list('Models','MAPE','MAE','RMSE'))

TypeError: unhashable type: 'list'

## Plots

In [ ]:
# Reversing Z-score normalization

casted_mean = india_cases_mean.to_numpy()
casted_std = india_cases_std.to_numpy()

india_cases_test_scaled = (india_cases_std[0]*india_cases_test)+india_cases_mean[0]
yhat_uni_non_stacked_india_scaled = (india_cases_std[0]*yhat_uni_non_stacked_india)+india_cases_mean[0]
yhat_uni_non_stacked_long_india_scaled = (india_cases_std[0]*yhat_uni_non_stacked_long_india)+india_cases_mean[0]
yhat_uni_stacked_india_scaled = (india_cases_std[0]*yhat_uni_stacked_india)+india_cases_mean[0]
yhat_uni_stacked_long_india_scaled = (india_cases_std[0]*yhat_uni_stacked_long_india)+india_cases_mean[0]

usa_cases_test_scaled = (usa_cases_std[0]*usa_cases_test)+usa_cases_mean[0]
yhat_uni_non_stacked_usa_scaled = (usa_cases_std[0]*yhat_uni_non_stacked_usa)+usa_cases_mean[0]
yhat_uni_non_stacked_long_usa_scaled = (usa_cases_std[0]*yhat_uni_non_stacked_long_usa)+usa_cases_mean[0]
yhat_uni_stacked_usa_scaled = (usa_cases_std[0]*yhat_uni_stacked_usa)+usa_cases_mean[0]
yhat_uni_stacked_long_usa_scaled = (usa_cases_std[0]*yhat_uni_stacked_long_usa)+usa_cases_mean[0]

print(yhat_uni_non_stacked_india_scaled.size)
print(india_cases_test_scaled.size)

In [ ]:
def plot_fore_test(test, fore, title):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(12, 8)

    ax.plot(test, color='blue', label='Test')
    ax.plot(fore, color='red', label='Forecast')
    ax.legend(loc='best')
    plt.title(title)
    plt.show()

### Univariate Non Stacked -- Rolling

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_non_stacked_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_non_stacked_usa_scaled, title='Daily cases')

### Univariate Non Stacked -- Long-Term

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_non_stacked_long_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_non_stacked_long_usa_scaled, title='Daily cases')

### Univariate Stacked -- Rolling

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_stacked_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_stacked_usa_scaled, title='Daily cases')

### Univariate Stacked -- Long-Term

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_stacked_long_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_stacked_long_usa_scaled, title='Daily cases')

#  Evaluation Metrics

In [ ]:
keras_mape = MeanAbsolutePercentageError()
keras_mape(india_cases_test_scaled[3:], yhat_uni_non_stacked_india_scaled).numpy()

In [ ]:
def MAPE(Y_actual,Y_Predicted, title):
    mask = Y_actual != 0
    
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual)[mask])*100
    print(f"MAPE of {title} is {mape}%")
    
mape_non_stacked_india = MAPE(india_cases_test_scaled[3:], yhat_uni_non_stacked_india_scaled, title="India Non-stacked")
mape_stacked_india = MAPE(india_cases_test_scaled[3:], yhat_uni_stacked_india_scaled, title="India Stacked")
# mape_vacc = MAPE(val['Confirmed'], roll_fc, title="Daily Cases")

In [ ]:
mape_non_stacked_usa = MAPE(usa_cases_test_scaled[3:], yhat_uni_non_stacked_usa_scaled, title="USA Non-stacked")
mape_stacked_usa = MAPE(usa_cases_test_scaled[3:], yhat_uni_stacked_usa_scaled, title="USA Stacked")

In [ ]:
from sklearn.metrics import mean_absolute_error

print('MAE of USA Non-stacked:', mean_absolute_error(usa_cases_test_scaled[3:], yhat_uni_non_stacked_usa_scaled))
print('MAE of USA Stacked:', mean_absolute_error(usa_cases_test_scaled[3:], yhat_uni_stacked_usa_scaled))

In [ ]:
print('MAE of India Non-stacked:', mean_absolute_error(india_cases_test_scaled[3:], yhat_uni_non_stacked_india_scaled))
print('MAE of Inida stacked:', mean_absolute_error(india_cases_test_scaled[3:], yhat_uni_stacked_india_scaled))